In [ ]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import torch
import torchvision.models as models
import cv2
from torchvision import datasets, transforms
import time

In [ ]:
from torchvision.models import resnet50
import torch.nn.functional as F

class EmbeddingResnet(nn.Module):
	def __init__(self):
		super(EmbeddingResnet, self).__init__()
		
		resnet = resnet50(pretrained=True)
		self.features = nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool, resnet.layer1, resnet.layer2, resnet.layer3, resnet.layer4, resnet.avgpool)
		#self.features = nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool, resnet.layer1,  
		#	nn.Dropout(0.5), resnet.layer2, nn.Dropout(0.5), resnet.layer3, nn.Dropout(0.5), resnet.layer4, nn.Dropout(0.5),
 		#	resnet.avgpool)
		self.act = nn.ReLU6(inplace=True)
		# Fix blocks
		for p in self.features[0].parameters(): p.requires_grad=False
		for p in self.features[1].parameters(): p.requires_grad=False
		for p in self.features[6].parameters(): p.requires_grad=True
		for p in self.features[5].parameters(): p.requires_grad=True
		for p in self.features[4].parameters(): p.requires_grad=False
		self.act.requires_grad = True

		def set_bn_fix(m):
			classname = m.__class__.__name__
			if classname.find('BatchNorm') != -1:
				for p in m.parameters(): p.requires_grad=False

		self.features.apply(set_bn_fix)

	def forward(self, x):
		features = self.features.forward(x)
		features = features.view(features.size(0), -1)
		#features = F.normalize(features, p=2, dim=1)
		return self.act(features)

class EmbeddingDensenet(nn.Module):
    def __init__(self):
        super(EmbeddingDensenet, self).__init__()
        self.densenet = torch.hub.load('pytorch/vision:v0.5.0', 'densenet121', pretrained=True)
        #self.seq = torch.nn.Sequential(*list(densenet.children())[:-1])
        for p in self.densenet.features:
            p.requires_grad = True

    def forward(self, x):
        features = torch.nn.Sequential(*(list(self.densenet.children())[:-1]))
        features = features(x)
        features = features.view(features.size(0), -1)
        features = F.normalize(features, p=2, dim=1)
        return features


In [ ]:
from torch import nn
from torch.nn import functional as F
from torch.nn import init
import torchvision

# Open Reid
class ResNet(nn.Module):
    __factory = {
        18: torchvision.models.resnet18,
        34: torchvision.models.resnet34,
        50: torchvision.models.resnet50,
        101: torchvision.models.resnet101,
        152: torchvision.models.resnet152,
    }

    def __init__(self, depth, pretrained=True, cut_at_pooling=False,
                 num_features=0, norm=False, dropout=0, num_classes=0):
        super(ResNet, self).__init__()

        self.depth = depth
        self.pretrained = pretrained
        self.cut_at_pooling = cut_at_pooling

        # Construct base (pretrained) resnet
        if depth not in ResNet.__factory:
            raise KeyError("Unsupported depth:", depth)
        self.base = ResNet.__factory[depth](pretrained=pretrained)

        if not self.cut_at_pooling:
            self.num_features = num_features
            self.norm = norm
            self.dropout = dropout
            self.has_embedding = num_features > 0
            self.num_classes = num_classes

            out_planes = self.base.fc.in_features

            # Append new layers
            if self.has_embedding:
                self.feat = nn.Linear(out_planes, self.num_features)
                self.feat_bn = nn.BatchNorm1d(self.num_features)
                init.kaiming_normal(self.feat.weight, mode='fan_out')
                init.constant(self.feat.bias, 0)
                init.constant(self.feat_bn.weight, 1)
                init.constant(self.feat_bn.bias, 0)
            else:
                # Change the num_features to CNN output channels
                self.num_features = out_planes
            if self.dropout > 0:
                self.drop = nn.Dropout(self.dropout)
            if self.num_classes > 0:
                self.classifier = nn.Linear(self.num_features, self.num_classes)
                init.normal(self.classifier.weight, std=0.001)
                init.constant(self.classifier.bias, 0)

        if not self.pretrained:
            self.reset_params()

    def forward(self, x):
        for name, module in self.base._modules.items():
            if name == 'avgpool':
                break
            x = module(x)

        if self.cut_at_pooling:
            return x

        x = F.avg_pool2d(x, x.size()[2:])
        x = x.view(x.size(0), -1)

        if self.has_embedding:
            x = self.feat(x)
            x = self.feat_bn(x)
        if self.norm:
            x = F.normalize(x)
        elif self.has_embedding:
            x = F.relu(x)
        if self.dropout > 0:
            x = self.drop(x)
        if self.num_classes > 0:
            x = self.classifier(x)
        return x

    def reset_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant(m.weight, 1)
                init.constant(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant(m.bias, 0)

def resnet50(**kwargs):
    return ResNet(50, **kwargs)


In [ ]:
class TripletNet(nn.Module):
    def __init__(self, embeddingNet):
        super(TripletNet, self).__init__()
        self.embeddingNet = embeddingNet

    def forward(self, i1, i2, i3):
        E1 = self.embeddingNet(i1)
        E2 = self.embeddingNet(i2)
        E3 = self.embeddingNet(i3)
        # dist_a = F.pairwise_distance(E1, E2, 2)
        # dist_b = F.pairwise_distance(E1, E3, 2)
        return E1, E2, E3

In [ ]:
class TripletNetInference(TripletNet):
    def __init__(self, embeddingNet):
        super(TripletNetInference, self).__init__(embeddingNet)
    
    def forward(self, x):
        return self.embeddingNet(x)

In [ ]:
# Open Reid
ckp = torch.load('../open-reid/examples/logs/model_best.pth.tar')
model = resnet50(num_features=1024,num_classes=4096)
model.load_state_dict(ckp['state_dict'])
#model.cut_at_pooling = True
model = nn.DataParallel(model)
model = model.to('cuda')
model.eval()

In [ ]:
# encoding: utf-8
"""
@author:  liaoxingyu
@contact: sherlockliao01@gmail.com
"""

import math

import torch
from torch import nn
import torchvision.models as models


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, last_stride=2, block=Bottleneck, layers=[3, 4, 6, 3]):
        self.inplanes = 64
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        # self.relu = nn.ReLU(inplace=True)   # add missed relu
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        #self.dropout = nn.Dropout(0.5)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.dropout1 = nn.Dropout(0.5)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.dropout2 = nn.Dropout(0.5)
        self.layer4 = self._make_layer(
            block, 512, layers[3], stride=last_stride)
        self.dropout3 = nn.Dropout(0.5)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        # x = self.relu(x)    # add missed relu
        x = self.maxpool(x)

        x = self.layer1(x)
        #x = self.dropout(x)
        x = self.layer2(x)
        x = self.dropout1(x)
        x = self.layer3(x)
        x = self.dropout2(x)
        x = self.layer4(x)
        x = self.dropout3(x)

        return x

    def load_param(self, model_path):
        param_dict = torch.load(model_path)
        for i in param_dict:
            if 'fc' in i:
                continue
            self.state_dict()[i].copy_(param_dict[i])

    def random_init(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()



def weights_init_kaiming(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight, a=0, mode='fan_out')
        nn.init.constant_(m.bias, 0.0)
    elif classname.find('Conv') != -1:
        nn.init.kaiming_normal_(m.weight, a=0, mode='fan_in')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)
    elif classname.find('BatchNorm') != -1:
        if m.affine:
            nn.init.constant_(m.weight, 1.0)
            nn.init.constant_(m.bias, 0.0)


def weights_init_classifier(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.normal_(m.weight, std=0.001)
        if m.bias:
            nn.init.constant_(m.bias, 0.0)


class Baseline(nn.Module):
    in_planes = 2048

    def __init__(self, num_classes, last_stride, model_path, neck, neck_feat, model_name, pretrain_choice):
        super(Baseline, self).__init__()
        if model_name == 'resnet18':
            self.in_planes = 512
            self.base = ResNet(last_stride=last_stride, 
                               block=BasicBlock, 
                               layers=[2, 2, 2, 2])
        elif model_name == 'resnet34':
            self.in_planes = 512
            self.base = ResNet(last_stride=last_stride,
                               block=BasicBlock,
                               layers=[3, 4, 6, 3])
        elif model_name == 'resnet50':
            self.base = ResNet(last_stride=last_stride,
                               block=Bottleneck,
                               layers=[3, 4, 6, 3])
        elif model_name == 'resnet101':
            self.base = ResNet(last_stride=last_stride,
                               block=Bottleneck, 
                               layers=[3, 4, 23, 3])
        elif model_name == 'resnet152':
            self.base = ResNet(last_stride=last_stride, 
                               block=Bottleneck,
                               layers=[3, 8, 36, 3])
            
        elif model_name == 'se_resnet50':
            self.base = SENet(block=SEResNetBottleneck, 
                              layers=[3, 4, 6, 3], 
                              groups=1, 
                              reduction=16,
                              dropout_p=None, 
                              inplanes=64, 
                              input_3x3=False,
                              downsample_kernel_size=1, 
                              downsample_padding=0,
                              last_stride=last_stride) 
        elif model_name == 'se_resnet101':
            self.base = SENet(block=SEResNetBottleneck, 
                              layers=[3, 4, 23, 3], 
                              groups=1, 
                              reduction=16,
                              dropout_p=None, 
                              inplanes=64, 
                              input_3x3=False,
                              downsample_kernel_size=1, 
                              downsample_padding=0,
                              last_stride=last_stride)
        elif model_name == 'se_resnet152':
            self.base = SENet(block=SEResNetBottleneck, 
                              layers=[3, 8, 36, 3],
                              groups=1, 
                              reduction=16,
                              dropout_p=None, 
                              inplanes=64, 
                              input_3x3=False,
                              downsample_kernel_size=1, 
                              downsample_padding=0,
                              last_stride=last_stride)  
        elif model_name == 'se_resnext50':
            self.base = SENet(block=SEResNeXtBottleneck,
                              layers=[3, 4, 6, 3], 
                              groups=32, 
                              reduction=16,
                              dropout_p=None, 
                              inplanes=64, 
                              input_3x3=False,
                              downsample_kernel_size=1, 
                              downsample_padding=0,
                              last_stride=last_stride) 
        elif model_name == 'se_resnext101':
            self.base = SENet(block=SEResNeXtBottleneck,
                              layers=[3, 4, 23, 3], 
                              groups=32, 
                              reduction=16,
                              dropout_p=None, 
                              inplanes=64, 
                              input_3x3=False,
                              downsample_kernel_size=1, 
                              downsample_padding=0,
                              last_stride=last_stride)
        elif model_name == 'senet154':
            self.base = SENet(block=SEBottleneck, 
                              layers=[3, 8, 36, 3],
                              groups=64, 
                              reduction=16,
                              dropout_p=0.2, 
                              last_stride=last_stride)
        elif model_name == 'resnet50_ibn_a':
            self.base = resnet50_ibn_a(last_stride)
        elif model_name == 'densenet':
            self.base = models.densenet121(pretrained=True)
            self.base.classifier = nn.Linear(1024, self.in_planes)
            
        if pretrain_choice == 'imagenet':
            self.base.load_param(model_path)
            print('Loading pretrained ImageNet model......')

        self.gap = nn.AdaptiveAvgPool2d(1)
        # self.gap = nn.AdaptiveMaxPool2d(1)
        self.num_classes = num_classes
        self.neck = neck
        self.neck_feat = neck_feat

        if self.neck == 'no':
            self.classifier = nn.Linear(self.in_planes, self.num_classes)
            # self.classifier = nn.Linear(self.in_planes, self.num_classes, bias=False)     # new add by luo
            # self.classifier.apply(weights_init_classifier)  # new add by luo
        elif self.neck == 'bnneck':
            self.bottleneck = nn.BatchNorm1d(self.in_planes)
            self.bottleneck.bias.requires_grad_(False)  # no shift
            self.classifier = nn.Linear(self.in_planes, self.num_classes, bias=False)

            self.bottleneck.apply(weights_init_kaiming)
            self.classifier.apply(weights_init_classifier)

    def forward(self, x):

        global_feat = self.gap(self.base(x))  # (b, 2048, 1, 1)
        #global_feat = self.base(x)
        global_feat = global_feat.view(global_feat.shape[0], -1)  # flatten to (bs, 2048)
        if not self.training:
            return global_feat

        if self.neck == 'no':
            feat = global_feat
        elif self.neck == 'bnneck':
            feat = self.bottleneck(global_feat)  # normalize for angular softmax

        if self.training:
            cls_score = self.classifier(feat)
            return cls_score, global_feat  # global feature for triplet loss
        else:
            if self.neck_feat == 'after':
                # print("Test with feature after BN")
                return feat
            else:
                # print("Test with feature before BN")
                return global_feat

    def load_param(self, trained_path):
        param_dict = torch.load(trained_path)
        for k, v in param_dict.state_dict().items():
            if 'classifier' in k:
                continue
            self.state_dict()[k].copy_(param_dict.state_dict()[k])



In [ ]:
model = models.resnet50(pretrained=True)
model = nn.DataParallel(model)
model = model.to('cuda')
model.eval()

In [ ]:
ckp = torch.load('../pytorch-siamese-triplet/data/resnet_triplet_big/checkpoint_30.pth')
embeddingNet = EmbeddingResnet()
resnet = TripletNetInference(embeddingNet)
model = nn.DataParallel(resnet)
model = model.to('cuda')
model.load_state_dict(ckp['state_dict'])
model.eval()

In [ ]:
#ckp = torch.load('amur/model/checkpoint_5.pth')
model_dict = ckp['state_dict']
embeddingNet = EmbeddingResnet()
#embeddingNet = EmbeddingDensenet()
model_dict_mod = {}
for key, value in model_dict.items():
    new_key = '.'.join(key.split('.')[2:])
    #print(new_key)
    model_dict_mod[new_key] = value
model = embeddingNet.to('cuda')
#model.load_state_dict(model_dict_mod)

In [ ]:
from object_detection import ObjectDetector

In [ ]:
detector = ObjectDetector('ssd/saved_model')
detector.loadModel()
detector.getBoundingBoxes(cv2.imread('amur_detection_small\\0042.jpg'))

In [ ]:
import os

# Read in all ELP images
img_folder = 'amur/plain_reid_tf/'
train_classes = [x for x in os.listdir(os.path.join(img_folder, 'train'))]
val_classes = [x for x in os.listdir(os.path.join(img_folder, 'valid'))]

x_train_names = []
y_train = []
x_val_names = []
y_val = []

for cls in train_classes:
    base_path = os.path.join(img_folder, os.path.join('train', cls))
    imgs = [x for x in os.listdir(base_path) if x.endswith('.jpg')]
    if len(imgs) < 3:
        print('Skipping class {}'.format(cls))
        continue
    for img in imgs:
        x_train_names.append(os.path.join(base_path, img))
        y_train.append(cls)

for cls in val_classes:
    base_path = os.path.join(img_folder, os.path.join('valid', cls))
    imgs = [x for x in os.listdir(base_path) if x.endswith('.jpg')]
    for img in imgs:
        x_val_names.append(os.path.join(base_path, img))
        y_val.append(cls)

print('Train classes: {}, Valid Classes: {}'.format(len(set(y_train)), len(set(y_val))))
print('Total Train images: {}, Total validation images: {}'.format(len(x_train_names), len(x_val_names)))

In [ ]:
import sys
sys.path.insert(0, '../reid-strong-baseline')

#modelpath = 'trained_models/elp_no_ls_12_ins_resnet50_model_80.pth'
modelpath = 'trained_models/amur_no_ls_8_ins_99_acc_resnet50_model_100.pth'
# Strong Reid
#ckp = torch.load('../reid-strong-baseline/amur_test1/resnet50_checkpoint_850.pth')
#model = Baseline(107,1,'C:\\Users\\Prashanth\\.cache\\torch\\checkpoints\\resnet50-19c8e357.pth', 'bnneck', 'after', 'resnet50', 'self')
#model = torch.load('../reid-strong-baseline/amur_test1/resnet50_model_120.pth')
#model = torch.load('../reid-strong-baseline/jag_test/resnet50_model_120.pth')
#model = torch.load('../reid-strong-baseline/elp_test/resnet50_model_60.pth')
#model = torch.load('trained_models/jag_no_ls_8_ins_resnet50_model_80.pth')
#model = torh.load('trained_models/amur_99.9_strong_reid_good_resnet50_model_50.pth')
model = torch.load(modelpath, torch.device('cpu'))
#model.cut_at_pooling = True
model = nn.DataParallel(model)
model = model.to('cpu')
model.eval()

In [ ]:
import os
import numpy as np

# Read in all ELP images
#img_folder = 'ELPephants/reid_faces_renamed/test'
#img_folder = 'ELPephants/reid_faces_renamed/test/'
#img_folder = 'jaguars/reid_strong_baseline/train'
img_folder = 'amur/plain_reid_train/train/'
images = [x for x in os.listdir(img_folder) if x.endswith('.jpg')]

with open(os.path.join(img_folder, 'class_mapping.txt')) as f:
    mapping = {x.split('\t')[0].strip() : x.split('\t')[1].strip() for x in f.readlines()}

mapping = {os.path.join(img_folder, k):v for k,v in mapping.items() if k in images}

rev_map = dict()
for k,v in mapping.items():
    if v not in rev_map:
        rev_map[v] = []
    rev_map[v].append(k)

rev_map = {k:v for k,v in rev_map.items() if len(v) > 1}

x_train_names = []
y_train = []
x_val_names = []
y_val = []

for k,v in rev_map.items():
    # 30% split of each class to training and test
    num_el = int(np.ceil(len(v) * 0.3))
    val = np.random.choice(v, num_el, replace=False)
    train = [x for x in v if x not in val]
    x_train_names.extend(train)
    x_val_names.extend(val)
    y_train.extend([k for _ in range(len(train))])
    y_val.extend([k for _ in range(len(val))])
    assert len(x_train_names) == len(y_train)
    assert len(x_val_names) == len(y_val)

def shuffle(x,y):
    c = list(zip(x,y))
    np.random.shuffle(c)
    return zip(*c)

# Verify correctness of data
for index in range(len(x_val_names)):
    assert mapping[x_val_names[index]] == y_val[index], '{} != {}'.format(mapping[x_val_names[index]],y_val[index])
    
for index in range(len(x_train_names)):
    assert mapping[x_train_names[index]] == y_train[index]
    
# Map to congiguous identities
num_ids = len(set(y_train))
num_t_ids = len(set(y_val))

assert num_ids == num_t_ids

id_map = dict()
counter = 0
for id in y_train:
    if not id in id_map:
        id_map[id] = counter
        counter += 1

rev_id_map = {v:k for k,v in id_map.items()}
        
y_train_old, y_val_old = y_train, y_val
y_train = np.array([id_map[x] for x in y_train])
y_val = np.array([id_map[x] for x in y_val])

for i in range(len(y_train)):
    assert y_train_old[i] == rev_id_map[y_train[i]]

for i in range(len(y_val)):
    assert y_val_old[i] == rev_id_map[y_val[i]]

x_train_names, y_train = shuffle(x_train_names, y_train)
x_val_names, y_val = shuffle(x_val_names, y_val)
y_train = np.array(y_train)
y_val = np.array(y_val)

print('Train classes: {}, Valid Classes: {}'.format(len(set(y_train)), len(set(y_val))))
print('Total Train images: {}, Total validation images: {}'.format(len(x_train_names), len(x_val_names)))

In [ ]:
def extract_feature(img):
    img = cv2.resize(img, (256,256))
    img = img[:,:,(2,1,0)]
    transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                           ])
    img = transform(img)
    img = img.reshape((1, 3, 256,256))
    with torch.no_grad():
        model.eval()
        img = torch.autograd.Variable(img).cuda()
        res = model(img)
        #res = torch.sigmoid(res)
        return res.cpu().detach().numpy().flatten()

In [ ]:
import sys
sys.path.insert(0, '../reid-strong-baseline')

mdl = Baseline(len(set(y_train)), 1, modelpath, 'bnneck', 'after', 'resnet50', 'self')
mdl.load_param(modelpath)


model = mdl
model = nn.DataParallel(model)
model = model.to('cuda')
model.eval()

In [ ]:
x_train.shape

In [ ]:
# Extract feature for each image in training set
start = time.time()
x_train = np.asarray([extract_feature(cv2.imread(x)) for x in x_train_names])
x_val = np.asarray([extract_feature(cv2.imread(x)) for x in x_val_names])
total_time = time.time() - start
print('Total FE time: {:.4f}s, Average time per image: {:.4f}s'.format(
    total_time, total_time/(len(x_train) + len(x_val))))

In [ ]:
from sklearn import svm
from sklearn import metrics

def createSvmModel(kernelType='linear'):
    if 'poly' == kernelType:
        return svm.SVC(kernel=kernelType, gamma='scale', degree=2)
    return svm.SVC(kernel=kernelType, gamma='scale')

In [ ]:
svmModel = createSvmModel()
svmModel = svmModel.fit(x_train, y_train)
#svmModel = svmModel.fit(x_val, y_val)
#svmModel = svmModel.fit(x_train_new, y_train_new)

y_pred = svmModel.predict(x_train)
acc = metrics.accuracy_score(y_train, y_pred)
print('Training Accuracy: {:.4f}'.format(acc))

y_pred = svmModel.predict(x_val)
acc = metrics.accuracy_score(y_val, y_pred)
print('Validation Accuracy: {:.4f}'.format(acc))

In [ ]:
im1 = cv2.imread('videos/tiger_1.jpg')
im2 = cv2.imread('videos/tiger_2.jpg')

In [ ]:
im1 = ('videos/tiger_1.jpg')
im2 = ('videos/tiger_2.jpg')

In [ ]:
torch.sigmoid(torch.tensor(x_train[0]))

In [ ]:
def get_bb_from_image(imname, detector):
    img = cv2.imread(imname)
    bounding_boxes = detector.getBoundingBoxes(img)
    if len(bounding_boxes) == 1:
        box = bounding_boxes[0].bounding_box
        img = img[box.ymin:box.ymax, box.xmin:box.xmax,:]
    return img

In [ ]:
# Extract feature for each image in training set
x_train = np.asarray([extract_feature(get_bb_from_image(x, detector)) for x in x_train_names])
x_val = np.asarray([extract_feature(get_bb_from_image(x, detector)) for x in x_val_names])

In [ ]:
mapping = dict()
for k,v in zip(x_val_names, y_val):
    if not v in mapping:
        mapping[v] = []
    mapping[v].append(k)
#mapping = {k:v for k,v in mapping.items() if len(v) > 2}

In [ ]:
import matplotlib.pyplot as plt
from scipy import spatial

In [ ]:
#im1 = mapping[9][2]
#im2 = mapping[8][0]
im1 = 'images/reid/cub_2.png'
im2 = 'images/reid/cub_4.png'
img1 = cv2.imread(im1)
img2 = cv2.imread(im2)

#img1 = get_bb_from_image(im1, detector)
#img2 = get_bb_from_image(im2, detector)
f1 = extract_feature(img1)
f2 = extract_feature(img2)

fig,ax = plt.subplots(1,2)
ax[0].imshow(img1)
ax[1].imshow(img2)
print(np.linalg.norm(f1 - f2))
print('Cos Sim: {:.4f}'.format(1 - spatial.distance.cosine(f1, f2)))

In [ ]:
mapping.keys()

In [ ]:
distmap = dict()
for i in range(len(y_val)):
    if not y_val[i] in distmap:
        distmap[y_val[i]] = []
        
    distmap[y_val[i]].append(x_val[i])

In [ ]:
from scipy.spatial import distance_matrix

distmat = distance_matrix(x_train_t, x_train_t)
indices = np.argsort(distmat, axis=1)

In [ ]:
matches = (y_train[indices] == (y_train)[:, np.newaxis]).astype(np.int32)

In [ ]:
for i in range(len(y_train)):
    cmc = matches[i]
    print(cmc.cumsum())

In [ ]:
indices[0] == np.array(sorted(y_train[:,np.newaxis]))

In [ ]:
y_train[indices[1]][:10]

In [ ]:
y_train[1]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
#cosine = cosine_similarity(x_train, x_train)

In [ ]:
xtrainlist = x_train.tolist()
xvallist = x_val.tolist()
ytrainlist = list(y_train)
yvallist = list(y_val)
for im in os.listdir('images/reid'):
    xtrainlist.append(extract_feature(cv2.imread(os.path.join('images/reid', im))))
    if 'cub' in im:
        ytrainlist.append(1000)
    else:
        ytrainlist.append(1001)

xvallist.append(xtrainlist[-1])
yvallist.append(yvallist[-1])
xtrainlist = xtrainlist[:-1]
ytrainlist = ytrainlist[:-1]

x_train_new = np.array(xtrainlist)
y_train_new = np.array(ytrainlist)
x_val_new = np.array(xvallist)
y_val_new = np.array(yvallist)

In [ ]:
x_val = np.asarray([extract_feature(cv2.imread(x)) for x in x_val_names])

In [ ]:
x_train1 = []
x_val1 = []
for i in range(len(x_train)):
    x_train[i][x_train[i] == 0] = 0.000001

for i in range(len(x_val)):
    x_val[i][x_val[i] == 0] = 0.000001

In [ ]:
x_train1 = np.log(x_train1)
x_val1 = np.log(x_val1)

In [ ]:
x_train,y_train = shuffle(x_train, y_train)

In [ ]:
distr = []
for k,v in mapping.items():
    distr.append((k,len(v)))

print(sorted(distr, key=lambda x: -x[1]))

In [ ]:
x_train.shape

In [ ]:
def predict(imname, model):
    img = cv2.imread(imname)
    feat = np.array([extract_feature(img)])
    res = model.predict(feat)
    return res

In [ ]:
predict(im2, svmModel)

In [ ]:
y_pred = svmModel.predict(x_train_new)
acc = metrics.accuracy_score(y_train_new, y_pred)
print('Training Accuracy: {:.4f}'.format(acc))

y_pred = svmModel.predict(x_val_new)
acc = metrics.accuracy_score(y_val_new, y_pred)
print('Validation Accuracy: {:.4f}'.format(acc))

In [ ]:
predict('jaguars/reid_for_tensorflow/31/j128_28_0.jpg', svmModel)

In [ ]:
predict('elp_small/373_Ariel II right side_26Jan2016.jpg', svmModel)

In [ ]:
image = 'amur/plain_reid_train/train/000037.jpg'
if image in mapping:
    print('Original: {}'.format(mapping[image]))
image = 'images/reid/cub_5.png'
num = predict(image, svmModel)
print(num)
#plt.imshow(cv2.imread(image))
#showAllTigers(rev_map, num, img_folder)

In [ ]:
image = 'jaguars/reid_strong_baseline/test/j128_0_0.jpg'
if image in mapping:
    print('Original: {}'.format(mapping[image]))
#image = 'images/reid/tiger_3.png'
num = predict(image, svmModel)
print(num)
#plt.imshow(cv2.imread(image))
#showAllTigers(rev_map, num, img_folder)

In [ ]:
x_train_names[0]

In [ ]:
mapping

In [ ]:
def showAllTigers(rev_map, num, folder):
    files = np.array(rev_map[str(num)])
    size = 2 * (files.size//2)
    files = files[:size]
    files = files.reshape(2, files.size//2)
    fig, axes = plt.subplots(files.shape[0], files.shape[1], figsize=(24,3))
    m,n = files.shape
    for i in range(m):
        for j in range(n):
            image = cv2.imread(files[i][j])
            if image is None:
                print('None!')
                continue
            axes[i][j].imshow(image)
            axes[i][j].axis('off')

In [ ]:
showAllTigers(rev_map, 1, img_folder)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
def createTsne(x_train, y_train, title):
    fig, ax = plt.subplots()

    X_embedded = TSNE(n_components=2).fit_transform(x_train)
    x, y = X_embedded[:,0], X_embedded[:,1]
    colors = plt.cm.rainbow(np.linspace(0,1,10))
    sc = ax.scatter(x, y, c=[int(x) for x in y_train], cmap=matplotlib.colors.ListedColormap(colors))
    plt.colorbar(sc)
    ax.set_title(title)
    ax.axis('off')

In [ ]:
newx = np.concatenate((x_train, x_val), axis=0)
newy = np.concatenate((y_train, y_val), axis=0)

In [ ]:
#createTsne(newx, newy, 'Elephants - Feature Vector projections over Test set')
#createTsne(x_train, y_train, 'Tiger - Feature Vector projections over Train set')
createTsne(x_val, y_val, 'Tiger - Feature Vector projections over Validation set')

In [ ]:
fig, ax = plt.subplots()

X_embedded = TSNE(n_components=2).fit_transform(x_train)
x, y = X_embedded[:,0], X_embedded[:,1]
colors = plt.cm.rainbow(np.linspace(0,1,10))
sc = ax.scatter(x, y, c=[int(x) for x in y_train], cmap=matplotlib.colors.ListedColormap(colors))
plt.colorbar(sc)
plt.show()

In [ ]:
fig, ax = plt.subplots()
X_embedded = TSNE(n_components=2).fit_transform(x_val)
x, y = X_embedded[:,0], X_embedded[:,1]
colors = plt.cm.rainbow(np.linspace(0,1,10))
sc = ax.scatter(x, y, c=[int(x) for x in y_val], cmap=matplotlib.colors.ListedColormap(colors))
plt.colorbar(sc)
plt.show()

In [ ]:
fig, ax = plt.subplots()
X_embedded = TSNE(n_components=2).fit_transform(x_train1)
x, y = X_embedded[:,0], X_embedded[:,1]
colors = plt.cm.rainbow(np.linspace(0,1,10))
sc = ax.scatter(x, y, c=[int(x) for x in y_train], cmap=matplotlib.colors.ListedColormap(colors))
plt.colorbar(sc)
plt.show()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=58, random_state=0).fit(x_train)

In [ ]:
y_pred = [str(x) for x in kmeans.predict(x_val)]
acc = metrics.accuracy_score(y_val, y_pred)
print(acc)

In [ ]:
model = models.densenet201(pretrained=True)
model.classifier = nn.Linear(in_features=1920, out_features=2048)
model.to('cuda')

In [ ]:
from torchsummary import summary
summary(model, (3, 256,256))

In [ ]:
for p in mdl.features:
    p.requires_grad = True

In [ ]:
def showImagesByIndex(indices, nameslist, targetlist, qidx, valnames, valtgt):
    files = np.array(nameslist)[indices]
    crct = (np.array(targetlist)[indices] == valtgt[qidx])
    size = 2 * (files.size//2)
    files = files[:size]
    files = files.reshape(2, files.size//2)
    tgtlist = np.array(targetlist)[indices].reshape(2, files.size//2)
    crct = crct.reshape(2, files.size//2)
    m,n = files.shape
    w,h = 300,300
    thickness = 6
    #fig, axes = plt.subplots(m+1, n, figsize=(6,6))
    fig, axes = plt.subplots(m+1, n)

    ax = axes[0][0]
    ax.imshow(cv2.imread(valnames[qidx][:,:,(2,1,0)]))
    ax.axis('off')
    ax.set_title(valtgt[qidx])
    fig.tight_layout()
    for i in range(n):
        axes[0][i].axis('off')

    for i in range(m):
        for j in range(n):
            image = cv2.imread(files[i][j])
            image = cv2.resize(image, (w,h))
            if crct[i][j]:
                image = cv2.rectangle(image, (thickness,thickness), (w-thickness,h-thickness), color=(0,255,0), thickness=thickness)
            else:
                image = cv2.rectangle(image, (thickness,thickness), (w-thickness,h-thickness), color=(0,0,255), thickness=thickness)
            if image is None:
                print('None!')
                continue
            axes[i+1][j].imshow(image[:,:,(2,1,0)])
            axes[i+1][j].axis('off')
            axes[i+1][j].set_title(tgtlist[i][j])
    fig.tight_layout(pad=0, w_pad=0, h_pad=0)
    #fig,ax = fig.subplots(1, 1, figsize=(12,6))

In [ ]:
def showSixImagesByIndex(indices, nameslist, targetlist, qidx, valnames, valtgt, rank=5):
    m = len(qidx)
    n = rank + 1 # first image is query
    #fig, axes = plt.subplots(m, n, figsize=(30,15))
    fig, axes = plt.subplots(m, n, figsize=(12,12))
    w,h = 300,300
    thickness = 6
        
    for i in range(m):
        ax = axes[i][0]
        img = cv2.imread(valnames[qidx[i]])[:,:,(2,1,0)]
        img = cv2.resize(img, (w,h))
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(valtgt[qidx[i]])
        
        files = np.array(nameslist)[indices[i]]
        crct = (np.array(targetlist)[indices[i]] == valtgt[qidx[i]])
        tgtlist = np.array(targetlist)[indices[i]]
        for j in range(1,n):
            image = cv2.imread(files[j-1])
            image = cv2.resize(image, (w,h))
            if crct[j-1]:
                image = cv2.rectangle(image, (thickness,thickness), (w-thickness,h-thickness), color=(0,255,0), thickness=thickness)
            else:
                image = cv2.rectangle(image, (thickness,thickness), (w-thickness,h-thickness), color=(0,0,255), thickness=thickness)
            if image is None:
                print('None!')
                continue
            axes[i][j].imshow(image[:,:,(2,1,0)])
            axes[i][j].axis('off')
            axes[i][j].set_title(tgtlist[j-1])
    #fig.subplots_adjust(hspace=0)
    #fig.tight_layout(pad=-0.9)
    fig.subplots_adjust(wspace=.1, hspace=0)
    #fig,ax = fig.subplots(1, 1, figsize=(12,6))

In [ ]:
# Calculating accuracy of identification - Qualitative
from scipy.spatial import distance_matrix

distmat = distance_matrix(x_val, x_train)
m, n = distmat.shape

In [ ]:
rank = 5
numimages = 5
resindices = []
resqidx = []
ids = np.random.choice(list(range(0,len(set(y_val)))), numimages, replace=False)
id_map = dict()
for i in range(len(y_val)):
    if y_val[i] not in ids:
        continue

    x = y_val[i]
    if not x in id_map:
        id_map[x] = []
    id_map[x].append(i)

#for x in range(numimages):
for x in ids:
    i = np.random.choice(id_map[x], replace=False)
    #i = ids[x]
    resqidx.append(i)
    distances = distmat[i]
    indices = np.argsort(distances)[:rank]
    resindices.append(indices)
    org_classes = y_train[indices]
    org_class = y_val[i]
    matches = (org_class == org_classes).astype(int)
    cmc = matches.cumsum()
    cmc[cmc > 1] = 1
    print('Top-{} Rank: {:.4f}'.format(rank, cmc.sum()/len(cmc)))
    print('Total correct identities: {}/{}'.format(matches.sum(), rank))

showSixImagesByIndex(resindices, x_train_names, y_train, resqidx, x_val_names, y_val)

In [ ]:
m,n

In [ ]:
d = distmat[0]
indices = np.argsort(d)[:10]
org_classes = y_train[indices]
org_class = y_val[0]
mat = (org_class == org_classes).astype(int)

In [ ]:
mat = mat.cumsum()
mat[mat > 1] = 1

In [ ]:
(mat == 1).sum() / len(mat)

In [ ]:
np.sort(d)

In [ ]:
showImagesByIndex(org_classes, x_train_names, y_train, 0, x_val_names, y_val)